# Исследование взаимовлияния скважин

## CRMIP модель

CRM или емкостно-резистивная модель основывается на уравнении материального баланса и формуле Дюпюи позволяя быстро получать значения межскважинной интерференции.
Теория модели берет свое начало из электротехники и утверждает, что "выходная характеристика системы в первую очередь зависит от входного сигнала и сопротивления в системе".


## Импорт необходимых библиотек

В данном проекте я постарался исползовать минимум сторонних библиотек и ограничеться только одной дополнительной SciPy, чтобы использовать метод оптимизации (минимизации разницы и поиска коэффициентов для самой модели).

In [1]:
import math
import scipy
from openpyxl import load_workbook

### Объявляем необходимые переменные для работы

In [2]:
wb = load_workbook(r"technical_data2.xlsx")
sheet = wb.worksheets[0] # Атрибут для книги эксель
inj_amount = 0 # Количество нагнетательных скважин
prod_amount = 0 # Количество добывающих скважин
f = {} # Коэффициент взаимовлияния
tau = {} # Коэффициент временного лага
qliq0 = {} # Начальные дебиты
inj_wells_rates = {} # Словарь ключи имена скважин, значения данные по закачке # TODO: Можно оставить как локальну переменную в методе
prod_wells_rates = {} # Данные по добыче # TODO: Можно  оставить как локальную переменную в методе
prod_CRM = {} # Об
MSE = {}
sum_prod_CRM = {} # Сумма всех CRM моделей (равна количеству добывающих скважин)
time = [] # Время, в целом оно не нужно - есть количество строк, то есть шагов "К"

### Функция распределение параметров и значений по словорям

In [3]:
#def amount_of_params(sheet, inj_amount, prod_amount, prod_wells_rates, inj_wells_rates): # TODO: Мне кажется всю эту функцию можно оставить в init
for i in range(2, sheet.max_row):
    time.append(sheet[i][0].value)
for column in range(1, sheet.max_column):
    q_data = [] #временная переменная для добавления расходов доб/нагн скважин
    # Идем по листу и добавляем информацию в словари
    if 'INJ' in sheet[1][column].value:
        inj_amount += 1
        for i in range(2, sheet.max_row):
            q_data.append(sheet[i][column].value)
        inj_wells_rates.update({sheet[1][column].value: q_data})
    else:
        prod_amount += 1
        for i in range(2, sheet.max_row):
            q_data.append(sheet[i][column].value)
        prod_wells_rates.update({sheet[1][column].value: q_data})
        MSE.update({'MSE ' + sheet[1][column].value: None})
# Создаем словари с нужными параметрами и названиями CRM-скважин-моделей
for i in prod_wells_rates:
    for j in inj_wells_rates:
        f.update({'f_' + str(i[5::]) +  ' ' + str(j[4::]): None})
        tau.update({'tau_' + str(i[5::]) + ' ' + str(j[4::]): None})
        qliq0.update({'qliq0_' + str(i[5::]) +  ' ' + str(j[4::]): None})
        prod_CRM.update({i[5::] + ' & ' + j[4::]: []})
#return f

### Функция автоматического создания начальных параметров

In [14]:
for i in inj_wells_rates:
    print(i, inj_wells_rates[i], end='\n')

INJ PH7
INJ IH4


In [4]:
qliq0

{'qliq0_PH12 PH7': None,
 'qliq0_PH12 IH4': None,
 'qliq0_IH3 PH7': None,
 'qliq0_IH3 IH4': None}

In [5]:
def differ(list1, list2):
    MSE = [(i + j) ** 2 for i, j in zip(list1, list2)] # TODO: Разве тут не минус вместо плюса должен быть?
    return sum(MSE)/len(MSE)

In [6]:
qliq0, tau, f, sum_prod_CRM

({'qliq0_PH12 PH7': None,
  'qliq0_PH12 IH4': None,
  'qliq0_IH3 PH7': None,
  'qliq0_IH3 IH4': None},
 {'tau_PH12 PH7': None,
  'tau_PH12 IH4': None,
  'tau_IH3 PH7': None,
  'tau_IH3 IH4': None},
 {'f_PH12 PH7': None,
  'f_PH12 IH4': None,
  'f_IH3 PH7': None,
  'f_IH3 IH4': None},
 {})

In [9]:
tau_input = [500.01, 434.24, 116, 1111]

In [11]:
for i, j in enumerate(tau):
    # Значение по ключу j равно значению из тау инпут
    tau[j] = tau_input[i]

In [12]:
tau

{'tau_PH12 PH7': 500.01,
 'tau_PH12 IH4': 434.24,
 'tau_IH3 PH7': 116,
 'tau_IH3 IH4': 1111}

In [7]:
def crm_calculate(in_package):

    global qliq0

    global sum_prod_CRM

    list_length = len(qliq0)
    q_input = in_package[0:list_length]
    tau_input = in_package[list_length: list_length * 2]
    f_input = in_package[list_length * 2: list_length * 3]


    for i, j in enumerate(prod_CRM):
        prod_CRM[j].append(list(qliq0.values())[i]) #В пустой словарь добавляет начальные значения для последующего расчета

    for i, j in enumerate(tau):
        #Значение по ключу j равно значению из тау инпут
        tau[j] = list(tau.values())[i]

    for i, j in enumerate(f):
        f[j] = list(f.values())[i]
    # Расчет дебитов самих по всем трем переменным


    for well_i, tau_i, f_i in zip(prod_CRM, tau, f):
        for i in range(1, len(time)): # TODO: проверить по последнему значению входит ли
            prod_CRM[well_i].append(round(prod_CRM[well_i][i-1] * math.exp(-(i - (i-1))/tau[tau_i]) + (1 - math.exp(-(i - (i-1))/tau[tau_i])) * f[f_i], 1))
    sum_prod_CRM = sum_prod_CRM.fromkeys(list(prod_wells_rates.keys()), []) # Здесь просто создается словарь CRM моделей скважин по существующим реальным скважинам
    step = len(list(prod_CRM.values())) // prod_amount
    boundary2 = step
    boundary1 = 0
    #print(list(prod_CRM.keys()))
    for i in sum_prod_CRM:
        #print(list(prod_CRM.values())[boundary1:boundary2])
        sum_prod_CRM[i] = list(map(sum, zip(*list(prod_CRM.values())[boundary1:boundary2])))  # Здесь неправильно (диапазон по индексам листа) суммируются CRM модели добывающих скважин по нагнетательным (их же несколько) и получают одну CRM на одну добывающую. Суммируются шагом.
        boundary2 += step
        boundary1 += step

    for i, j, k in zip(MSE, sum_prod_CRM, prod_wells_rates):
        MSE[i] = round(differ(sum_prod_CRM[j], prod_wells_rates[k]), 3)

    return sum(MSE.values())

In [8]:
result = scipy.optimize.minimize(crm_calculate)

TypeError: minimize() missing 1 required positional argument: 'x0'